In [12]:
import pandas as pd
from itertools import combinations
from collections import Counter
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpInteger, LpStatus, value
import numpy as np

# 1. LOAD DATA
df = pd.read_csv('player_data_sample.csv')
players = list(df['player_code'])
role_map = df.set_index('player_code')['role'].to_dict()
target_perc = df.set_index('player_code')['perc_selection'].to_dict()
target_counts = {p: int(round(target_perc[p]*20000)) for p in players}

# 2. ENUMERATE VALID TEAMS (FAST)
role_players = {role: list(df[df['role'] == role]['player_code']) for role in df['role'].unique()}
all_teams = []
for team in combinations(players, 11):
    rc = Counter(role_map[p] for p in team)
    if all(rc[r] >= 1 for r in ['WK', 'Batsman', 'Bowler', 'Allrounder']):
        all_teams.append(team)
print(f"Enumerated {len(all_teams)} valid teams.")

# 3. SETUP INTEGER PROGRAM
prob = LpProblem("FantasyAssignment", LpMinimize)
team_vars = [LpVariable(f"team_{i}", 0, 20000, LpInteger) for i in range(len(all_teams))]
# For each player, actual count equals sum of team vars * indicator
actual_hits = {p: lpSum([team_vars[i] * int(p in all_teams[i]) for i in range(len(all_teams))]) for p in players}
# Add error variables for each player (abs)
abs_errors = []
for p in players:
    extra = LpVariable(f"above_{p}", 0, 20000, LpInteger)
    sub   = LpVariable(f"below_{p}", 0, 20000, LpInteger)
    prob += actual_hits[p] - target_counts[p] == extra - sub
    abs_errors.append(extra + sub)
# Constraint: total number of teams exactly 20000
prob += lpSum(team_vars) == 20000
# Objective: minimize total error
prob += lpSum(abs_errors)
print("Solving integer program. This may take 5-15 minutes...")

prob.solve()
print(f"Solver status: {LpStatus[prob.status]}")

# 4. EXTRACT AND WRITE TEAMS
final_teams = []
for i, v in enumerate(team_vars):
    count = int(round(value(v)))
    final_teams += [all_teams[i]]*count
print(f"Total teams built: {len(final_teams)}")

# 5. EVALUATE & SAVE OUTPUT
player_freq = Counter([p for team in final_teams for p in team])
eval_rows = []
for p in players:
    exp = target_perc[p]
    obs = player_freq[p]/20000
    err = abs(obs-exp)/exp if exp > 0 else 0
    flag = "✅" if err <= 0.05 else "❌"
    eval_rows.append({
        "player_code": p,
        "player_name": df[df['player_code']==p]['player_name'].values[0],
        "role": role_map[p],
        "target_frac": exp,
        "actual_frac": obs,
        "error_pct": err*100,
        "within_5%": abs(obs-exp)/exp <= 0.05,
    })
    print(f"{flag} {eval_rows[-1]['player_name']:<12} | {role_map[p]:<10} | "
          f"Target: {exp*100:6.2f}% | Actual: {obs*100:6.2f}% | Error: {err*100:6.2f}%")
passcount = sum(row['within_5%'] for row in eval_rows)
print(f"\nFINAL RESULT: {passcount}/22 within ±5%")
if passcount>=20:
    print("Assignment Passed! ✅")
else:
    print("Assignment Failed! ❌")

accuracy_df = pd.DataFrame(eval_rows)
accuracy_df.to_csv("accuracy_summary.csv", index=False)
# Save teams CSV
rows = []
for tid, team in enumerate(final_teams, 1):
    for p in team:
        info = df[df['player_code']==p].iloc[0]
        rows.append({
            'team_id': tid,
            'match_code': info['match_code'],
            'player_code': p,
            'player_name': info['player_name'],
            'role': info['role'],
            'team': info['team'],
            'perc_selection': info['perc_selection'],
        })
teamdf = pd.DataFrame(rows)
teamdf.to_csv('team_df.csv', index=False)
print("\nCSV output complete: team_df.csv, accuracy_summary.csv")

Enumerated 529112 valid teams.
Solving integer program. This may take 5-15 minutes...
Solver status: Optimal
Total teams built: 20000
✅ Player_20    | WK         | Target:  39.57% | Actual:  39.57% | Error:   0.00%
❌ Player_2     | WK         | Target:  21.31% | Actual:  71.67% | Error: 236.30%
✅ Player_7     | Allrounder | Target:  91.91% | Actual:  91.91% | Error:   0.00%
✅ Player_12    | Allrounder | Target:  95.22% | Actual:  95.22% | Error:   0.00%
✅ Player_10    | Allrounder | Target:  21.45% | Actual:  21.45% | Error:   0.00%
✅ Player_8     | Allrounder | Target:  56.82% | Actual:  56.82% | Error:   0.00%
✅ Player_22    | Allrounder | Target:  89.19% | Actual:  89.19% | Error:   0.00%
✅ Player_11    | Allrounder | Target:  27.09% | Actual:  27.09% | Error:   0.00%
✅ Player_21    | Batsman    | Target:  69.35% | Actual:  69.35% | Error:   0.00%
❌ Player_3     | Batsman    | Target:   2.83% | Actual:   7.58% | Error: 168.02%
✅ Player_18    | Batsman    | Target:  49.50% | Actual: 

In [22]:
# === ENSURE perc_selection IS PRESENT IN teamdf ===
if 'perc_selection' not in teamdf.columns:
    # Guaranteed merge from your original dataframe
    teamdf = teamdf.merge(df[['player_code', 'perc_selection']], on='player_code', how='left')

# === MANDATORY: ASSIGNMENT ACCURACY EVALUATION ===
def evaluate_team_accuracy(team_df):
    player_summary = team_df.drop_duplicates(['player_code', 'player_name', 'perc_selection'])
    total_teams = team_df['team_id'].nunique()
    team_counts = team_df['player_code'].value_counts().to_dict()

    accuracy_data = []
    for _, row in player_summary.iterrows():
        player_code = row['player_code']
        expected_perc = row['perc_selection']
        team_count = team_counts.get(player_code, 0)
        actual_perc = team_count / total_teams
        perc_error = (actual_perc - expected_perc) / expected_perc if expected_perc > 0 else 0
        accuracy_data.append({
            'player_code': player_code,
            'player_name': row['player_name'],
            'role': row['role'],
            'team': row['team'],
            'perc_selection': expected_perc,
            'team_count': team_count,
            'actual_perc_selection': actual_perc,
            'perc_error': perc_error,
            'within_5_percent': abs(perc_error) <= 0.05,
        })
    accuracy_df = pd.DataFrame(accuracy_data)
    accuracy_df.to_csv("accuracy_summary.csv", index=False)

    within_5 = accuracy_df['within_5_percent'].sum()
    print("\nAssignment Results:")
    print(f"Players within ±5%: {within_5}/{len(accuracy_df)}\n")
    print(f"{'Status':<3} {'Player':<12} | {'Role':<11} | Target   | Actual   |   Error")
    print("-"*55)
    for _, row in accuracy_df.iterrows():
        flag = "✅" if row['within_5_percent'] else "❌"
        print(f"{flag} {row['player_name']:<12} | {row['role']:<11} | "
              f"{row['perc_selection']*100:7.2f}% | {row['actual_perc_selection']*100:7.2f}% | {row['perc_error']*100:7.2f}%")
    print("\nAssignment " + ("PASSED ✅" if within_5 >= 20 else "FAILED ❌"))
    return accuracy_df

# === EVALUATION AND PRINTOUT ===
accuracy_df = evaluate_team_accuracy(teamdf)
print("\nAll outputs saved: team_df.csv, accuracy_summary.csv")

# === BONUS: INTERACTIVE PLOTLY CHART ===
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(go.Bar(
    x=accuracy_df['player_name'],
    y=accuracy_df['perc_selection']*100,
    name='Target %',
    marker_color='lightskyblue'
))
fig.add_trace(go.Bar(
    x=accuracy_df['player_name'],
    y=accuracy_df['actual_perc_selection']*100,
    name='Actual %',
    marker_color='seagreen'
))
fig.update_layout(
    barmode='group',
    title='Player Target vs. Actual Selection Percentage',
    xaxis_tickangle=-45,
    xaxis_title='Player',
    yaxis_title='Selection (%)',
    width=1000,
    height=500,
    legend_title="Legend"
)
fig.show()
fig.write_html("selection_distribution.html")
print("Bonus: Interactive Plotly selection_distribution.html saved!")



Assignment Results:
Players within ±5%: 20/22

Status Player       | Role        | Target   | Actual   |   Error
-------------------------------------------------------
✅ Player_20    | WK          |   39.57% |   39.57% |    0.00%
❌ Player_2     | WK          |   21.31% |   71.67% |  236.30%
✅ Player_7     | Allrounder  |   91.91% |   91.91% |    0.00%
✅ Player_12    | Allrounder  |   95.22% |   95.22% |    0.00%
✅ Player_8     | Allrounder  |   56.82% |   56.82% |    0.00%
✅ Player_22    | Allrounder  |   89.19% |   89.19% |    0.00%
✅ Player_21    | Batsman     |   69.35% |   69.35% |    0.00%
❌ Player_3     | Batsman     |    2.83% |    7.58% |  168.02%
✅ Player_4     | Batsman     |   65.90% |   65.90% |    0.00%
✅ Player_17    | Batsman     |   93.60% |   93.60% |    0.00%
✅ Player_1     | Bowler      |   66.34% |   66.34% |    0.00%
✅ Player_13    | Bowler      |   10.68% |   10.68% |    0.00%
✅ Player_5     | Bowler      |   83.48% |   83.48% |    0.00%
✅ Player_15    | Bowler 

Bonus: Interactive Plotly selection_distribution.html saved!
